# Sudoku generator

This is a python program that generates a sudoku for you to solve. You can choose the difficulty level (1- easy, 6-difficult) where you get level 3 by default.

Source: https://github.com/Periculum/SudokuGenerator


### Approach

We want the puzzle to only have one solution which is hard to generate.

There are multiple approaches possible. This program first generates a fully solved sudoku. Then it removes a number and checks if there still is only one solution. It repeats this last step until enough numbers are removed.


To keep overview all functions are in a Sudoku class and are called by the main() method.

In [4]:
import sys
import random
import time



class Sudoku:

  def __init__(self):
    """
    The __init__() function is called automatically every time the class is being used to create a new object.
    """
    self.reset() # call reset function


  def reset(self):
    """
    Function that creates 9x9 two-dimensional array with zeros
    """
    rows=9
    columns=9
    self.board=[[0 for j in range(columns)] for i in range(rows)]


  def print(self):
    """
    Prints the board
    Replaces zeros for dots for more overview
    and adds spaces between the numbers
    """
    for i in range(9):
      print(" ".join([str(x) if x != 0 else "." for x in self.board[i]]))


  def number_is_valid(self, row, column, num):
    """
    Checks if num can be filled in certain cell [r][c]
    First, it checks if number is already in row or column.
    """
    for i in range(9):
      if self.board[row][i] == num or self.board[i][column] == num:
        return False
    """
    Then it checks if num already appears in the 3x3 sudoku box.
    We do this by first finding the coresponding coordinates of the upper-left of the sudoku box.
    Note: // divides with integral result (discard remainder)
    So when column = 8 and row = 5, then start_column=6 and start_row=3.
    Then we check if num appears in the box.
    """
    start_column = column // 3 * 3
    start_row = row //3 * 3
    for i in range(3):
      for j in range(3):
        if self.board[i+start_row][j+start_column] == num:
          return False
    """
    If num does not appear in row, column or 3x3 box, then the number is valid
    and it returns True.
    """
    return True


  def solve(self):
    """
    The solver tries to find all possible solutions for a given board.

    The idea of the solve function is that it loops through all empty cells of the board
    and tries to find a valid number using the number_is_valid function. This solver
    is a backtracking-algorithm, meaning that if a mistake is made not it doesn't start over
    but it just removes the last few steps.

    Steps:
    - It loops through all cells of the sudoku and tries to find an empty cell.
    - It uses the number_is_valid function to find valid numbers for an empty cell
    - When it finds a valid number it is added to the board
    - Then the solve() function is called within its own function to continue with
    the updated board and find and fill in the next empty cell.
    - when it come back after the solve function it sets the cell to 0 and goes to the
    next number in the for-loop

    Note: The yield statement returns a generator object to the one who calls the function
    which contains yield, instead of simply returning a value.

    Note: "yield from self.solve()" sends the generated board back into the solve()
    function.
    """

    # look for empty cell in sudoku board
    for r in range(9):
      for c in range(9):
        if self.board[r][c] ==0:

          # find a valid number to fill in
          for n in range(1,10):
            if self.number_is_valid(r,c,n):

              self.board[r][c]=n
              yield from self.solve() # continue with updated board

              self.board[r][c]=0 # after solving, we clear so we can try the next number
          return
    yield True


  def evaluate(self, difficulty):
    """
    This function determines the level of difficulty of the sudoku by determining
    how many cells of the sudoku should empty.
    Input should be number between 1 (easy) and 6 (difficult)
    Fun fact: there must be at least 17 numbers in the sudoku puzzle for it to
    have at most one solution.
    """
    empty_cells = [0, 25, 35, 45, 52, 58, 64]
    if difficulty <0 or difficulty > len(empty_cells)-1:
      print('invalid difficulty', file=sys.stderr)
    return empty_cells[difficulty]


  def generate(self, difficulty):
    """
    First it fills the empty board to have a completed sudoku. Then it empties
    cells untill the chosen difficulty is reached. Then we empty cells one by one
    until there are enough, while checking if the sudoku still has one solution.

    We shouldn't only use the solve() function to fill the board because then
    we end up with the same sudoku all the time.

    The board is filled by first randomly filling the diagonal 3x3 boxes (because they
    don't affect each other) and then the solve() function is used to fill the board.

    Since we only need one solution and not all possible solutions we stop solve()
    after first succes.
    """
    for i in range(0,9,3): # gives[0, 3, 6]
      square = [1, 2, 3, 4, 5, 6, 7, 8, 9] # numbers to be filled out in each 3x3 box on the diagonal
      random.shuffle(square) # put entries of square in random order
      for r in range(3):
        for c in range(3):
          self.board[r+i][c+i] = square.pop() # .pop() gives the last element of list and removes it from that list

    for solutions in self.solve():
      break # stop after finding first solution

    # find amount of cells we need to empty
    empty_cells = self.evaluate(difficulty)

    # create a list of coordinates to visit and shuffle them
    unvisited = [(r, c) for r in range(9) for c in range(9)]
    random.shuffle(unvisited)

    while empty_cells > 0 and len(unvisited) >0: # continues until enough empty cells or all coordinates visited once
      r, c = unvisited.pop() # random coordinate
      copy = self.board[r][c] # save copy of value in that cell
      self.board[r][c] = 0 # empty this cell

      # find solutions for new board
      solutions = [solution for solution in self.solve()]

      if len(solutions) > 1: # if there is more than one solution for this board
        self.board[r][c] = copy # put back the number in that cell
      else: # if there is only one solution for this board then
        empty_cells -=1 # reduce empty_cell by 1 and continue

    # if unvisited is empty, but empty_cells not -> trying again
    if empty_cells > 0:
      print("No Sudoku found. Trying again.")
      return False
    else:
      return True







def main():
    """
    The main function in which we describe what needs to happen
    """

    # if difficulty isn't given as input the default should be 3
    # args = [int(x) if x.isdecimal() else x for x in sys.argv[1:]]
    # difficulty = args[0] if len(args)>0 else 3

    difficulty=2

    sudoku = Sudoku()

    # when the difficulty level is high it can take a long time to find a sudoku
    # therefore we set a timer for how long the program gets to find a solution
    timeout = 600 # 10 minutes
    start_time = time.time()
    end_time = start_time + timeout
    while time.time() < end_time:
      if sudoku.generate(difficulty) == True: # if a solution is found then stop
        break
      else:
        sudoku.reset() # no solution found then start again with empty sudoku

    # printing
    sudoku.print()






if __name__ == "__main__":
    main()


5 3 8 1 2 . 6 9 .
. 2 6 . 7 9 . . 4
. 7 9 . 3 6 2 1 .
. 8 7 . 4 5 9 . 1
6 9 . . . . . . 8
2 . 1 9 8 . 4 . 3
8 6 2 4 5 . . 7 9
9 . . . . . 1 . .
7 1 . 2 . . . . 6


## Testing the solver

In [24]:
### Testing the solver 

sudoku = Sudoku() # sudoku is object of class Sudoku

sudoku.board # Dit is the sudoku board made (because reset() is in the __init__ function)

# Change the board to some thing you want to solve

sudoku.board =  [[0, 0, 3, 0, 7, 0, 0, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 8, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 3],
 [0, 0, 0, 7, 0, 0, 0, 0, 4],
 [0, 0, 0, 2, 0, 0, 0, 0, 0],
 [5, 0, 6, 0, 0, 0, 0, 0, 9],
 [9, 0, 0, 0, 4, 0, 0, 0, 5],
 [0, 8, 0, 3, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 8, 0, 0, 0]]

# use solve until solution is found
for solutions in sudoku.solve():
      break

# show solution
sudoku.board

[[4, 2, 3, 8, 7, 5, 9, 6, 1],
 [1, 6, 9, 4, 2, 3, 5, 8, 7],
 [7, 5, 8, 9, 6, 1, 4, 2, 3],
 [8, 1, 2, 7, 3, 9, 6, 5, 4],
 [3, 9, 4, 2, 5, 6, 7, 1, 8],
 [5, 7, 6, 1, 8, 4, 2, 3, 9],
 [9, 3, 1, 6, 4, 2, 8, 7, 5],
 [6, 8, 5, 3, 9, 7, 1, 4, 2],
 [2, 4, 7, 5, 1, 8, 3, 9, 6]]